# 000 Forecasting Bot

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IFP import IFP, list_questions

In [3]:
ifps = list_questions()['results']

In [5]:
today_ids = list(sorted([x['id'] for x in ifps])) if ifps else [27238, 27239, 27240, 27241]
today_ids

[28006, 28007, 28008]

In [6]:
ifps = {id: IFP(id) for id in today_ids}

In [7]:
from Forecaster import Forecaster

In [8]:
fcst = Forecaster(ifps)

In [10]:
fcst.fit()

MODEL DOMAIN
28006 Will Kamala Harris say "I'm speaking" in the presidential debate with Donald Trump?
Election outcome
28007 Will the national price of gasoline exceed $3.25 per gallon when accessed by Metaculus Admins on October 1, 2024?
Market price
28008 Will the US weekly 30-year fixed mortgage rate be above 6.35% on October 1, 2024, according to Freddie Mac?
Rate change
CORRELATOR
NEWS
RATES
FORECASTING National gasoline price on October 1, 2024
28007 25 Current trends indicate a steady decline in gas prices, supported by factors such as lower demand and decreasing oil prices. Based on recent historical trends and the calculated rate of change, it is expected that the national average price will be around $3.138 per gallon by October 1, 2024. Hence, there is a relatively low probability that it will exceed $3.25 per gallon.
FORECASTING US mortgage rate on October 1, 2024
28008 25 The US weekly 30-year fixed mortgage rate is currently at 6.35% and has been decreasing steadily. Wit

In [11]:
for ifp in ifps.values():
    try:
        print(ifp.id, ifp.forecast)
    except:
        print(ifp.id, 'none')

28006 85
28007 25
28008 25


In [12]:
fcst.report()

28006 Will Kamala Harris say "I'm speaking" in the presidential debate with Donald Trump?
Forecast 85
Rationale Kamala Harris has a history of utilizing the phrase "I'm speaking" in debates, and given the planned debate format with Trump, it seems likely she may use it again to assert herself or make a point. The microphone rules also suggest situations where she might need to use the phrase due to disruptions despite the rules. The probability reflects a high likelihood considering her past behavior and the circumstances of the debate. 

28007 Will the national price of gasoline exceed $3.25 per gallon when accessed by Metaculus Admins on October 1, 2024?
Forecast 25
Rationale Current trends indicate a steady decline in gas prices, supported by factors such as lower demand and decreasing oil prices. Based on recent historical trends and the calculated rate of change, it is expected that the national average price will be around $3.138 per gallon by October 1, 2024. Hence, there is a r

In [13]:
fcst.upload()

Prediction posted for  28006
Comment posted for  28006
Prediction posted for  28007
Comment posted for  28007
Prediction posted for  28008
Comment posted for  28008
